In [2]:
import torch
import pandas as pd 
import numpy as np

🔹 What’s left / next steps

Class distribution / skewness Done 

Check label counts → compute % per class Done

Merge similar attacks (e.g., SSH + FTP Bruteforce → “Bruteforce”) Done

Feature scaling

Neural networks are sensitive to numeric ranges

Apply StandardScaler or MinMaxScaler to all numeric columns

Categorical encoding

Protocol → one-hot or label encoding

Dst_Port → optional: one-hot top N ports, merge rare ports into “other”

Train/test split

Stratified by Label

Recommended: 80% train / 20% test

Class imbalance handling

Optional but recommended:

Use class weights in NN loss function

Or oversample rare attack classes

Optional feature engineering / selection

Check correlation → remove extremely redundant features

Create engineered temporal features from Timestamp if desired (hour, day of week)

Final sanity checks

No NaNs, inf, or placeholder negatives remain

All features are numeric and scaled

Categorical features encoded

Labels merged/encoded and consistent

In [3]:
df=  pd.read_csv(r"/root/Cyber/Cyber_security/datasets/02-14-2018_uncleaned.csv")

In [4]:
df.Label.value_counts(normalize=True)*100


Label
benign            63.544031
bruteforce        18.503838
ssh-bruteforce    17.952131
Name: proportion, dtype: float64

In [5]:
df["Label"]=df["Label"].replace(["ssh-bruteforce"],"bruteforce")

In [6]:
df.Label.value_counts(normalize=True)*100


Label
benign        63.544031
bruteforce    36.455969
Name: proportion, dtype: float64

WE WILL USE STANDARD SCALER BECAUSE MOST OF  THE PAPERS HAVE USED STANDARD SCALER WHICH USES Z SCORE NORMALIZATION
AND BECAUSE NETWORK TRAFFIC CAN HAVE VERY HIGH OUTLIERS AND MIN MAX SQUASHES THEM INTO A TINY RANGE , BEFORE SCALING WE NEED TO SPLIT THE DATA INTO TRAIN TEST SPLIT DUE TO DATA LEAKAGE

In [7]:
K = 10
top_ports = df["Dst_Port"].value_counts().nlargest(K).index

df["Dst_Port"] = df["Dst_Port"].where(
    df["Dst_Port"].isin(top_ports),
    "Other"
)

df["Dst_Port"] = df["Dst_Port"].astype(str)


In [8]:
print(df["Dst_Port"].value_counts())

Dst_Port
53       198117
21       193422
22       187881
80       129625
Other     93741
443       92094
3389      90606
445       40304
0         11876
5355       5083
137        2191
Name: count, dtype: int64


In [9]:
df["Label"]=df["Label"].str.strip().str.lower()

In [10]:
df["Label"].value_counts()

Label
benign        663997
bruteforce    380943
Name: count, dtype: int64

In [11]:
df['Label_encoded'] = df['Label'].map({'benign': 0, 'bruteforce': 1})

In [12]:
df['Label']=df["Label_encoded"]
df.drop(columns=["Label_encoded"],inplace=True)
df["Label"].value_counts()

Label
0    663997
1    380943
Name: count, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer


In [14]:
onehot_cols = ["Protocol", "Dst_Port"]

numeric_cols = df.drop(columns=["Label", "Protocol", "Dst_Port"]).columns


In [15]:


preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("cat", OneHotEncoder(
            drop="first",          # avoids dummy trap
            handle_unknown="ignore",
            sparse_output=False
        ), onehot_cols)
    ],
    remainder="drop"
)


In [16]:
X = df.drop(columns=["Label"])
y = df["Label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)


In [17]:
#Sanity check
df["Dst_Port"].apply(type).value_counts()


Dst_Port
<class 'str'>    1044940
Name: count, dtype: int64

In [23]:
print(type(X_train_proc))
type(X_test_proc)

<class 'numpy.ndarray'>


numpy.ndarray

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


Checklist 

Understand & Prepare Data ✔️

Collect dataset, clean & preprocess ✔️

Split into train/validation/test sets  ✔️

Normalize/scale features ✔️

Batch the data

Define Model Architecture

Decide number & type of layers

Choose activation functions

Think about input & output shapes

Choose Loss Function & Optimizer

Pick a loss suitable for task (classification/regression)

Select optimizer (Adam, SGD, etc.)

Set learning rate & other hyperparameters

Train the Model

Loop over epochs and batches

Forward pass → compute loss → backward pass → update weights

Monitor training loss & metrics

Validate & Evaluate

Test on unseen data

Calculate accuracy, F1, RMSE, or other metrics

Adjust model or hyperparameters if needed

Save & Deploy

Save model weights

Load model for inference or further training

Iterate & Improve

Tune hyperparameters

Experiment with architecture

Handle overfitting (dropout, regularization)

Scale up data if possible

In [25]:
y_train1=y_train.to_numpy()
y_test1=y_test.to_numpy()

In [27]:
X_train=torch.from_numpy(X_train_proc)
X_test=torch.from_numpy(X_test_proc)
Y_test=torch.from_numpy(y_test1)
Y_train=torch.from_numpy(y_train1)
print(X_train)
print(X_test)
print(Y_train)
print(Y_test)


tensor([[-0.3221, -0.1169, -0.0593,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3234, -0.0722, -0.0309,  ...,  0.0000,  1.0000,  0.0000],
        [-0.3228, -0.1169, -0.0593,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.3234, -0.1169, -0.0593,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3234, -0.1169, -0.0593,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3235, -0.0945, -0.0593,  ...,  0.0000,  0.0000,  1.0000]],
       dtype=torch.float64)
tensor([[-2.4156e-01,  3.9714e-02, -2.5028e-03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-3.2346e-01, -1.1692e-01, -5.9349e-02,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-3.2346e-01, -1.1692e-01, -5.9349e-02,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 4.0432e+00,  2.1872e-01,  6.3818e-02,  ...,  0.0000e+00,
          1.0000e+00,  0.0000e+00],
        [ 1.9700e+00,  1.9634e-01,  7.3292e-02,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-

In [ ]:
from torch.utils.data import TensorDataset

num_epoch=30
batch_size=1024
lr= 1e-3
input_dim=X_train.shape[1]

train_dataset = TensorDataset(X_train, Y_train)
test_dataset  = TensorDataset(X_test, Y_test)


test_dataloader=DataLoader(test_dataset,batch_size=batch_size,shuffle=True)
train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=False)

70


In [36]:
train_features,train_labels=next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([1024, 70])
Labels batch shape: torch.Size([1024])


In [37]:
#Compute class weights and use in loss function 
num_pos=(Y_train==1).sum()
num_neg=(Y_train==0).sum()
pos_weight=num_neg/num_pos

In [38]:
criterion= nn.BCEWithLogitsLoss(pos_weight=pos_weight)

Baseline Model Architecture

Input: n_features

Input Layer

Linear(n_features → 128)

BatchNorm1d(128)

ReLU

Dropout(p = 0.3)

Hidden Layer 1

Linear(128 → 64)

BatchNorm1d(64)

ReLU

Dropout(p = 0.3)

Hidden Layer 2

Linear(64 → 32)

ReLU

Output Layer

Linear(32 → 1)

Sigmoid

In [ ]:
class BruteForceNN(nn.Module):
    def __init__(self,input_dim):
        super().__init__()
        